In [2]:
''' Soil water retention curve '''
from bokeh.embed import components
from bokeh.models.widgets import Slider

from bokeh.layouts import row, widgetbox
import numpy as np
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource

# Set up callbacks
def update_data(attrname, old, new):

    # Get the current slider values
    a = alpha.value
    b = n.value
    c = theta_r.value
    d = theta_s.value

    # Generate the new curve
    x = np.logspace(-3, 5, N)
    y = swrfn(x, a, b, c, d)

    source.data = dict(x=x, y=y)

def swrc_app():
    # Set up data
    swrfn = lambda psi, alpha, n, theta_r, theta_s: theta_r + (theta_s-theta_r)*((1+(alpha*psi)**(n))**(1-1/n))**(-1)
    N = 200
    x = np.logspace(-3, 5, N)
    alpha_start = 0.05
    n_start = 1.5
    theta_r_start = 0.05
    theta_s_start = 0.5
    y = swrfn(x,alpha_start, n_start, theta_r_start, theta_s_start)
    source = ColumnDataSource(data=dict(x=x, y=y))

    # Set up plot
    plot = figure(plot_height=400, plot_width=400, title="Soil water rentention curve",
                  tools="crosshair,pan,reset,save,wheel_zoom",
                  x_range=[0, 10**5], y_range=[0, 0.6],
                  x_axis_type="log")

    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


    # Set up widgets
    alpha = Slider(title="Alpha", value=alpha_start, start=0.01, end=2.0, step=0.01)
    n = Slider(title="n", value=n_start, start=1.2, end=3.0, step=0.1)
    theta_r = Slider(title="Θ_r", value=theta_r_start, start=0, end=0.15, step=0.01)
    theta_s = Slider(title="Θ_s", value=theta_s_start, start=0.35, end=0.6, step=0.01)
    
    for w in [alpha, n, theta_r, theta_s]:
        w.on_change('value', update_data)


    # Set up layouts and add to document
    inputs = widgetbox(alpha, n, theta_r, theta_s)

    #curdoc().add_root(row(inputs, plot, width=800))
    #curdoc().title = "swrc"
    plot = row(inputs, plot, width=800)

    script, div = components(plot)
    return script, div
